# Assembling

This notebook is used to assemble a processable h5ad object for subsequent notebooks.

## Fill in input data, output and settings

In [1]:
####################### TEST NAME ###################################
test = 'cropped_146'
################### DEFINING STRATEGY ###############################
from_h5ad = True #option 1
assembling_10_velocity = False #option 2
assembling_10_public = False #option 3
convert_seurat = False #option 4

###################### INPUT DATA ###################################

#For option 1: The path to an existing .h5ad file
h5ad_path = '/home/rstudio/data/anndata/cropped_146.h5ad'
#Manually set existing QC Columns
mean_insertsize = None
insertsize_count = None
n_total_fragments = None
n_fragments_in_promoters = None
pct_fragments_in_promotors = None
blacklist_overlaps = None
#total_number_of_fragments
TN = 'TN'
#uniquely_mapped_fragments
UM = 'UM'
#properly_paired_fragments
PP = 'PP'
#uniq_fragments
UQ = 'UQ'
#chrM_fragments
UM = 'UM'

#################### Calc QC Columns ###############################
#set available number of threads
n_threads = 8

#if the mean_insertsize should be calculated specify either a BAM file or a fragments file:
fragments_file = '/home/rstudio/data/bamfiles/fragments_cropped_146.bed'
bam_file = '/home/rstudio/data/bamfiles/sorted_cropped_146.bam'

#specify promotors_gtf if another promoter than the fixed ones should be used
#if promotors is None please specify species
#promoters_gtf =  '/mnt/flatfiles/organisms/new_organism/homo_sapiens/104/homo_sapiens.104.promoters2000.gtf'
promoters_gtf = '/home/rstudio/data/homo_sapiens.104.promoters2000.gtf'
species = None

##################### OUTPUT DATA ###################################
output_dir = '/home/rstudio/processed_data'

## Import modules

In [2]:
# sctoolbox modules 
import sctoolbox.atac_tree as sub_tree
import sctoolbox.creators as cr
import sctoolbox.fragment_length as fragments
import sctoolbox.atac as atac
import sctoolbox.calc_overlap_pct as overlap
from sctoolbox.qc_filter import *
from sctoolbox.atac_utils import *
# import episcanpy
import episcanpy as epi

## Setup path handling object 

In [3]:
# make an instance of the class
tree = sub_tree.ATAC_tree()
# set processing/output directory
tree.processing_dir = output_dir
# set sample/experiment.. 
tree.run = test

all directories existing
all directories existing


## Read in data

### Option 1: Read from .h5ad

In [4]:
qc_columns = {}
qc_columns["mean_insertsize"] = mean_insertsize
qc_columns['n_total_fragments'] = n_total_fragments
qc_columns['n_fragments_in_promoters'] = n_fragments_in_promoters
qc_columns['pct_fragments_in_promotors'] = pct_fragments_in_promotors
qc_columns["blacklist_overlaps"] = blacklist_overlaps
qc_columns["TN"] = TN
qc_columns["UM"] = UM
qc_columns["PP"] = PP
qc_columns["UQ"] = UQ
qc_columns["UM"] = UM

In [5]:
if from_h5ad:
    
    adata = epi.read_h5ad(h5ad_path)
    
    #Add information to the infoprocess
    cr.build_infor(adata, "Input_for_assembling", h5ad_path)
    cr.build_infor(adata, "Strategy", "Read from h5ad")
    
    print('add existing adata.obs columns to infoprocess:')
    print()
    for key, value in qc_columns.items():
        if value is not None:
            print(key + ':' + value)
            if value in adata.obs.columns:
                build_legend(adata, key, value)
            else:
                print('column:  ' + value + ' is not in adata.obs')
    
    if not adata.obs.index.name == "barcode":
        print('setting adata.obs.index = adata.obs[barcode]')
        adata.obs = adata.obs.set_index("barcode")
    else:
        print('barcodes are already the index')
    

add existing adata.obs columns to infoprocess:

TN:TN
UM:UM
PP:PP
UQ:UQ
setting adata.obs.index = adata.obs[barcode]


## Inspect adata

In [6]:
display(adata)

AnnData object with n_obs × n_vars = 373 × 3830
    obs: 'TN', 'UM', 'PP', 'UQ', 'CM', 'file', 'sample'
    var: 'name'
    uns: 'infoprocess', 'color_set', 'legend'

In [7]:
display(adata.var)

,name
0,b'chr1':9993-10421
1,b'chr1':29089-29368
2,b'chr1':180548-180799
3,b'chr1':199689-199928
4,b'chr1':629307-629902
...,...
3825,b'chrY':318869-319190
3826,b'chrY':386933-387197
3827,b'chrY':1453156-1453412
3828,b'chrY':1591522-1591761


In [8]:
display(adata.obs)

,TN,UM,PP,UQ,CM,file,sample
barcode,,,,,,,
AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/opt/project/data/snap/cropped_146.snap,MB
AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/opt/project/data/snap/cropped_146.snap,MB
AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/opt/project/data/snap/cropped_146.snap,MB
AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/opt/project/data/snap/cropped_146.snap,MB
AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/opt/project/data/snap/cropped_146.snap,MB
...,...,...,...,...,...,...,...
AAACTACCAGCTTGGCAACAGC,26.0,20.0,20.0,20.0,1.0,/opt/project/data/snap/cropped_146.snap,MB
AAACTACCAGCTTTAACTGCGC,29.0,20.0,20.0,19.0,0.0,/opt/project/data/snap/cropped_146.snap,MB
AAACTACCAGCTTTAGCCCTAT,32.0,24.0,24.0,24.0,1.0,/opt/project/data/snap/cropped_146.snap,MB


## Calculate missing columns

### 1. calc mean insertsize and count if missing 

In [9]:
if qc_columns['mean_insertsize'] is None or qc_columns['insertsize_count'] is None:
    atac.add_insertsize(adata, bam=bam_file)
    
    # update column dictionary
    qc_columns['mean_insertsize'] = 'mean_insertsize'
    qc_columns['insertsize_count'] = 'insertsize_count'
    
    # update infoprocess table
    build_legend(adata, 'mean_insertsize', 'mean_insertsize')
    build_legend(adata, 'insertsize_count', 'insertsize_count')
    
adata.obs

Opening bam file...
Creating chunks of size 100000bp...
Counting insertsizes across 30895 chunks...


Progress:   0%|          | 0/30895 [00:00<?, ?chunks/s]

Converting counts to dataframe
Done getting insertsizes from bam!
Added insertsize information to adata.obs[["insertsize_count", "mean_insertsize"]] and adata.uns["insertsize_distribution"].


,TN,UM,PP,UQ,CM,file,sample,insertsize_count,mean_insertsize
barcode,,,,,,,,,
AAACTACCAGAAACCCGAGATA,33.0,18.0,18.0,15.0,0.0,/opt/project/data/snap/cropped_146.snap,MB,44,90.91
AAACTACCAGAAACCTAAGTGG,52.0,36.0,35.0,32.0,1.0,/opt/project/data/snap/cropped_146.snap,MB,82,143.39
AAACTACCAGAAACGGATCAGT,27.0,19.0,19.0,19.0,0.0,/opt/project/data/snap/cropped_146.snap,MB,38,151.21
AAACTACCAGAAACGTCCCGTT,6223.0,5231.0,5213.0,3779.0,18.0,/opt/project/data/snap/cropped_146.snap,MB,11860,130.28
AAACTACCAGAAACTAGCCCTA,41.0,29.0,29.0,26.0,0.0,/opt/project/data/snap/cropped_146.snap,MB,60,113.80
...,...,...,...,...,...,...,...,...,...
AAACTACCAGCTTGGCAACAGC,26.0,20.0,20.0,20.0,1.0,/opt/project/data/snap/cropped_146.snap,MB,44,102.77
AAACTACCAGCTTTAACTGCGC,29.0,20.0,20.0,19.0,0.0,/opt/project/data/snap/cropped_146.snap,MB,42,124.00
AAACTACCAGCTTTAGCCCTAT,32.0,24.0,24.0,24.0,1.0,/opt/project/data/snap/cropped_146.snap,MB,54,117.11


### 2. promotor enrichment 

In [11]:
if qc_columns['n_fragments_in_promoters'] is None:
    overlap.pct_fragments_in_promoters(adata, promoters_gtf, species=species, bam_file=bam_file, cb_col=None, nproc=1)
    
    # n_total_fragments, n_fragments_in_promoters, pct_fragments_in_promoters
    # update column dictionary
    qc_columns['n_total_fragments'] = 'n_total_fragments'
    qc_columns['n_fragments_in_promoters'] = 'n_fragments_in_promoters'
    qc_columns['pct_fragments_in_promotors'] = 'pct_fragments_in_promotors'
    # update infoprocess table
    build_legend(adata, 'n_total_fragments', 'n_total_fragments')
    build_legend(adata, 'promotor_enrichment', 'promotor_enrichment')
    build_legend(adata, 'pct_fragments_in_promotors', 'pct_fragments_in_promotors')
    

Converting GTF to BED...
Converting BAM to fragments file! This may take a while...
Finished creating fragments file. Now sorting...
Finished sorting fragments
Finding overlaps...
Calculating percentage...
Adding results to adata object...


TypeError: Cannot setitem on a Categorical with a new category (0), set the categories first

## Inspect adata.obs

In [ ]:
adata.obs

## Save adata to .h5ad

In [ ]:
adata_output = tree.assembled_anndata
adata_output

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
adata.uns['insertsize_distribution'] = adata.uns['insertsize_distribution'].swapaxes('index', 'columns')

In [ ]:
adata.uns['insertsize_distribution'] = adata.uns['insertsize_distribution'].reset_index(drop=True)

In [ ]:
adata.uns['insertsize_distribution']

In [ ]:
#Saving the data
cr.build_infor(adata, "Test_number", test)
cr.build_infor(adata, "Anndata_path", output_dir)

adata_output = tree.assembled_anndata
adata.write(filename=adata_output)